# Load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

In [ ]:
import pandas as pd

result = pd.read_csv('/content/drive/My Drive/WorkOnMergedData/final_merged_data.csv')

In [ ]:
result.head()

,Unnamed: 0,Title,Post Author,Leading Comment,Reply Comments,Category,Forum
0,0,One item multiple buyers,Stocoloco_LLC,Yesterday I lowered the price of an item to ma...,['Isn’t that what you were trying to do??? Mat...,Fulfillment By Amazon,Amazon
1,1,Unavailable payment,storesupplies,I got my new credit card and before I could up...,"['You can’t. You just have to wait.', 'You can...",Fulfillment By Amazon,Amazon
2,2,Some FBA items Not Showing Inventory,ShipFlip,I sent an FBA shipment on November 26. They sh...,['This topic was automatically closed 180 days...,Fulfillment By Amazon,Amazon
3,3,Stock products,Andean_Star,"Hi, I need to know the products stock in Selle...",['You need to have a professional account and ...,Fulfillment By Amazon,Amazon
4,4,Frustrating Seller Support for faulty LTSF cat...,Illuminations_Brand,Just here to vent at the Asia based Seller Sup...,['Illuminations_Brand\nIf you’re here in the U...,Fulfillment By Amazon,Amazon


In [ ]:
result.isna().sum()

Unnamed: 0         0
Title              0
Post Author        0
Leading Comment    1
Reply Comments     0
Category           0
Forum              0
dtype: int64

In [ ]:
result.dropna(axis=0, inplace=True)

In [ ]:
result.columns

Index(['Unnamed: 0', 'Title', 'Post Author', 'Leading Comment',
       'Reply Comments', 'Category', 'Forum'],
      dtype='object')

In [ ]:
result.drop(columns=['Unnamed: 0', 'Post Author',
       'Reply Comments', 'Forum'], inplace=True)

In [ ]:
result.isna().sum()

Title              0
Leading Comment    0
Category           0
dtype: int64

In [ ]:
result.head()

,Title,Leading Comment,Category
0,One item multiple buyers,Yesterday I lowered the price of an item to ma...,Fulfillment By Amazon
1,Unavailable payment,I got my new credit card and before I could up...,Fulfillment By Amazon
2,Some FBA items Not Showing Inventory,I sent an FBA shipment on November 26. They sh...,Fulfillment By Amazon
3,Stock products,"Hi, I need to know the products stock in Selle...",Fulfillment By Amazon
4,Frustrating Seller Support for faulty LTSF cat...,Just here to vent at the Asia based Seller Sup...,Fulfillment By Amazon


In [ ]:
result.rename(columns={'Leading Comment': 'texts', 'Category': 'labels'}, inplace=True)

In [ ]:
result.labels.unique()

array(['Fulfillment By Amazon', 'Amazon Sponsored Products',
       'Account Health', 'Selling on Amazon', 'Global Selling', 'Groups',
       'Health,Safety,Sustainability,Security & Compliance',
       'Amazon Marketplace Web Service (MWS)', 'Login With Amazon',
       'Amazon Pay', 'Amazon Custom', 'US Announcements', 'Site Feedback',
       'Financial Management', 'Product Sourcing', 'Human Resources',
       'Amazon Specific', 'Flowster-specific', 'Software & Tools',
       'Fulfillment', 'Traffic Sources', 'Management', 'Misc Topics',
       'eCommerce Marketplaces', 'Store & Website Management'],
      dtype=object)

In [ ]:
len(result.labels.unique())

25

In [ ]:
categories = ['Fulfillment By Amazon', 'Amazon Sponsored Products',
       'Account Health', 'Selling on Amazon', 'Global Selling', 'Groups',
       'Amazon Marketplace Web Service (MWS)', 'Login With Amazon',
       'Amazon Pay', 'Amazon Custom', 'US Announcements', 'Site Feedback',
       'Health,Safety,Sustainability,Security & Compliance',
       'Financial Management', 'Product Sourcing', 'Human Resources',
       'Amazon Specific', 'Flowster-specific', 'Software & Tools',
       'Fulfillment', 'Traffic Sources', 'Management', 'Misc Topics',
       'eCommerce Marketplaces', 'Store & Website Management']

In [ ]:
len(categories)

25

# Train the model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

X = result.texts
y = result.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = 42)

In [ ]:
logreg = Pipeline([('vect', TfidfVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf', LogisticRegression(n_jobs=1, C=10.526405263157894, max_iter=200)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
res31 = accuracy_score(y_pred, y_test)
print(classification_report(y_test, y_pred))

accuracy 0.6628959276018099
                                                    precision    recall  f1-score   support

                                    Account Health       0.83      0.84      0.83       165
                                     Amazon Custom       0.93      0.41      0.57        32
              Amazon Marketplace Web Service (MWS)       0.80      0.62      0.70        26
                                        Amazon Pay       0.69      0.63      0.66        63
                                   Amazon Specific       0.00      0.00      0.00         4
                         Amazon Sponsored Products       0.84      0.70      0.76        30
                                 Flowster-specific       1.00      0.58      0.74        12
                                       Fulfillment       0.00      0.00      0.00         2
                             Fulfillment By Amazon       0.64      0.77      0.70       137
                                    Global Selling 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Run and test the recommender model

In [ ]:
# to generate the count matrix
count = TfidfVectorizer()
count_matrix = count.fit_transform(result['texts'])
count_matrix

<8840x24836 sparse matrix of type '<class 'numpy.float64'>'
	with 676684 stored elements in Compressed Sparse Row format>

In [ ]:
# to generate the cosine similarity matrix (size 250 x 250)
# rows represent all movies; columns represent all movies
# cosine similarity: similarity = cos(angle) = range from 0 (different) to 1 (similar)
# all the numbers on the diagonal are 1 because every movie is identical to itself (cosine value is 1 means exactly identical)
# matrix is also symmetrical because the similarity between A and B is the same as the similarity between B and A.
# for other values eg 0.1578947, movie x and movie y has similarity value of 0.1578947
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.04581401 0.06240643 ... 0.03376081 0.10718756 0.02796889]
 [0.04581401 1.         0.12163251 ... 0.04539971 0.06808399 0.03836429]
 [0.06240643 0.12163251 1.         ... 0.03232047 0.04969761 0.02816648]
 ...
 [0.03376081 0.04539971 0.03232047 ... 1.         0.08591856 0.76951469]
 [0.10718756 0.06808399 0.04969761 ... 0.08591856 1.         0.07868944]
 [0.02796889 0.03836429 0.02816648 ... 0.76951469 0.07868944 1.        ]]


In [ ]:
# to create a Series for movie titles which can be used as indices (each index is mapped to a movie title)
indices = pd.Series(result['Title'])
indices[:5]

0                             One item multiple buyers
1                                  Unavailable payment
2                 Some FBA items Not Showing Inventory
3                                       Stock products
4    Frustrating Seller Support for faulty LTSF cat...
Name: Title, dtype: object

In [ ]:
indices[indices=='One item multiple buyers']

0    One item multiple buyers
Name: Title, dtype: object

In [ ]:
# this function takes in a movie title as input and returns the top 10 recommended (similar) movies

def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]   # to get the index of the movie title matching the input movie
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   # similarity scores in descending order
    top_10_indices = list(score_series.iloc[1:11].index)   # to get the indices of top 10 most similar movies
    # [1:11] to exclude 0 (index 0 is the input movie itself)
    
    for i in top_10_indices:   # to append the titles of top 10 similar movies to the recommended_movies list
        recommended_movies.append(list(result['Title'])[i])
        
    return recommended_movies

recommend('Stock products')

['Buybox Seller Information',
 'Add product to my Listing through an ASIN using API',
 'In MWS, can you see if a product / offer is eligible for Prime shipping?',
 'Be in stock date',
 'Currently Unavailable',
 'Api for downloading product listing information and updating title and descriptions',
 'Shipping my inventory to FBA for the first time',
 'Stranded Inventory - Inactive (Out of Stock) for 3 weeks - Seller Support have been no help',
 'Shipping to closest distribution center',
 'API not passing through customer information']

In [ ]:
result[result['Title'] == 'About the Product Sourcing Category1']

,Title,texts,labels
8587,About the Product Sourcing Category1,Have questions about sourcing products? This i...,Product Sourcing


In [ ]:
recommend('About the Product Sourcing Category1')

['About the Fulfillment category1',
 'About the Management category1',
 'About the Amazon Specific Category1',
 'About the Software & Tools category1',
 'About the Store & Website Management category1',
 'About the Sales Channels & Marketplaces Category1',
 'About the Financial Management category1',
 'About the Human Resources category1',
 'About the Misc Topics category1',
 'About the Traffic Sources category1']

In [ ]:
print(result['texts'][result['Title'] == 'About the Product Sourcing Category1'])

8587    Have questions about sourcing products? This i...
Name: texts, dtype: object


In [ ]:
result['texts'][8587]

'Have questions about sourcing products? This is the category to use. Please be sure to select the most appropriate sub-category for your questions.'

In [ ]:
print(result['texts'][result['Title'] == 'About the Financial Management category1'])

8579    Have questions about Financial Management? Thi...
Name: texts, dtype: object


In [ ]:
result['texts'][8579]

'Have questions about Financial Management? This is the category to use. Please be sure to select the most appropriate sub-category for your questions.'

In [ ]:
print(result['texts'][result['Title'] == 'Shipment fulfillment'])

2266    Hello, Thank you for the answers and inputs in...
Name: texts, dtype: object


In [ ]:
result['texts'][2266]

'Hello, Thank you for the answers and inputs in advance. I have a question about fulfilling orders yourself.is there a cheap way other than going to the usps website. if you have experience in this please help out? thank you,'

In [ ]:
print(result['texts'][result['Title'] == 'About the Store & Website Management category1'])

8840    Have questions about Store & Website Managemen...
Name: texts, dtype: object


In [ ]:
result['texts'][8840]

'Have questions about Store & Website Management? This is the category to use. Please be sure to select the most appropriate sub-category for your questions.'